In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import numpy as np
import pandas as np
import physio # Toolbox made in Lyon, CRNL, France, for processing ECG and Respiratory signals
from configuration import *

from pycns import CnsReader, get_viewer # Toolbox made in Lyon, CRNL, France, for reading and viewing Moberg raw data

from physio_rea_analyse import * # Tools for computing binarized Heart / Resp rate variability
from custom_view import * # Tools for plotting 2D representations of Heart / Resp rate variability and others ...
from tools import * # useful tools

- pycns : https://github.com/samuelgarcia/pycns
- physio : https://github.com/samuelgarcia/physio

# Patient ID selection

In [3]:
patient_id = 'MF12'

# Metadata

In [4]:
meta = get_metadata(patient_id)
meta

sex                                              F
ddn                            1953-12-10 00:00:00
age                            1968-09-28 00:00:00
motif                                          HSA
entree_rea                     2022-09-08 00:00:00
debut_monito                   2022-09-09 00:00:00
fin_monito                     2022-09-15 00:00:00
durée                                          6.0
EEG                                            1.0
ECoG                                         strip
ptio2                                          0.0
PIC                                            1.0
NIRS                                           1.0
event_during_monito                            NaN
commentaire            SD; mort encephalique & NUP
Name: MF12, dtype: object

# Events

In [5]:
events = read_events(patient_id)
events['StartTimeGMT'] = pd.to_datetime(events['StartTime']).dt.tz_localize('Europe/Paris').dt.tz_convert('GMT')
events.head(10)

,Name,Description,StartTime,Duration,Target,EventDisplay,Source,StartTimeGMT
0,SD,None,2022-09-09 23:56:32.490,3806.2,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-09 21:56:32.490000+00:00
1,2:mydriase dt,None,2022-09-10 06:00:56.000,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS,2022-09-10 04:00:56+00:00
2,1:propofol,None,2022-09-10 06:24:46.233,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS,2022-09-10 04:24:46.233000+00:00
3,TDM,None,2022-09-10 07:56:50.279,2040.6,"EEG,Trends","{'@color': '#000000', '#text': 'Shade'}",User,2022-09-10 05:56:50.279000+00:00
4,SD,None,2022-09-10 14:34:24.651,872,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 12:34:24.651000+00:00
5,SD,None,2022-09-10 16:48:16.812,571.7,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 14:48:16.812000+00:00
6,SD,None,2022-09-10 18:50:23.445,582.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 16:50:23.445000+00:00
7,SD,None,2022-09-10 20:25:40.028,513.799,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 18:25:40.028000+00:00
8,SD,None,2022-09-10 21:24:27.666,658.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 19:24:27.666000+00:00
9,SD,None,2022-09-10 22:13:13.495,705.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 20:13:13.495000+00:00


# Preload data

In [6]:
path_to_data = data_path / patient_id # path to raw data
reader = CnsReader(path_to_data) # preload raw data

In [7]:
reader

CnsReader: MF12
17 streams : ['SpO2', 'CPP', 'ICP', 'EEG_NeonatalParams', 'ABP_Syst', 'HR', 'ECG_II', 'ABP', 'CO2', 'ABP_Mean', 'EEG', 'EEG_Impedance', 'Temp', 'EtCO2_Ext', 'ABP_Dias', 'PLETH', 'ICP_Mean']

# Detect ECG Peaks

In [8]:
ecg_stream = reader.streams['ECG_II']
srate_ecg = ecg_stream.sample_rate

start_date = None # from beginning
stop_date = None # to the end

ecg_raw, ecg_dates = ecg_stream.get_data(sel = slice(start_date,stop_date), with_times=True, apply_gain=False)
ecg, r_peaks = physio.compute_ecg(ecg_raw, srate_ecg, parameter_preset = 'human_ecg')
r_peaks['peak_date'] = ecg_dates[r_peaks['peak_index']]

In [9]:
r_peaks

,peak_index,peak_time,peak_date
0,164,0.328151,2022-09-09 17:16:04.809318
1,696,1.392639,2022-09-09 17:16:05.873776
2,1265,2.531162,2022-09-09 17:16:07.012265
3,1822,3.645674,2022-09-09 17:16:08.126743
4,2378,4.758185,2022-09-09 17:16:09.239221
...,...,...,...
514422,240161750,480544.165000,2022-09-15 07:17:07.974409
514423,240162146,480544.957364,2022-09-15 07:17:08.766771
514424,240162542,480545.749728,2022-09-15 07:17:09.559133
514425,240162937,480546.540091,2022-09-15 07:17:10.349494


# Detect Respiratory Cycles

In [10]:
co2_stream = reader.streams['CO2']
srate_co2 = co2_stream.sample_rate

start_date = None # from beginning
stop_date = None  # to the end

co2_raw, co2_dates = co2_stream.get_data(sel = slice(start_date,stop_date), with_times=True, apply_gain=False)

co2, resp_cycles = physio.compute_respiration(co2_raw, srate_co2, parameter_preset = 'human_co2')
resp_cycles['cycle_frequency_per_min'] = resp_cycles['cycle_freq'] * 60
resp_cycles['inspi_date'] = co2_dates[resp_cycles['inspi_index']]
resp_cycles['expi_date'] = co2_dates[resp_cycles['expi_index']]

In [11]:
resp_cycles

,inspi_index,expi_index,next_inspi_index,inspi_time,expi_time,next_inspi_time,cycle_duration,inspi_duration,expi_duration,cycle_freq,cycle_ratio,cycle_frequency_per_min,inspi_date,expi_date
0,512,709,1140,4.181027,5.789743,9.309318,5.128291,1.608715,3.519575,0.194997,0.313694,11.699804,2022-09-09 17:23:25.015960,2022-09-09 17:23:26.592637
1,1140,1349,1775,9.309318,11.016026,14.494771,5.185453,1.706708,3.478745,0.192847,0.329134,11.570830,2022-09-09 17:23:30.042118,2022-09-09 17:23:31.714837
2,1775,1976,2399,14.494771,16.136151,19.590398,5.095627,1.641380,3.454247,0.196247,0.322115,11.774803,2022-09-09 17:23:35.124301,2022-09-09 17:23:36.732992
3,2399,2703,3124,19.590398,22.072883,25.510798,5.920400,2.482485,3.437915,0.168908,0.419310,10.134451,2022-09-09 17:23:40.118446,2022-09-09 17:23:56.861637
4,3124,3331,3747,25.510798,27.201174,30.598258,5.087461,1.690376,3.397084,0.196562,0.332263,11.793703,2022-09-09 17:24:00.231085,2022-09-09 17:24:01.887796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119543,59982606,59982761,59983109,489822.065279,489823.331020,489826.172812,4.107532,1.265741,2.841792,0.243455,0.308151,14.607310,2022-09-15 07:16:53.283698,2022-09-15 07:16:54.524265
119544,59983109,59983259,59983595,489826.172812,489827.397722,489830.141521,3.968709,1.224910,2.743799,0.251971,0.308642,15.118265,2022-09-15 07:16:57.309537,2022-09-15 07:16:58.510086
119545,59983595,59983749,59984109,489830.141521,489831.399095,489834.338880,4.197359,1.257575,2.939785,0.238245,0.299611,14.294702,2022-09-15 07:17:01.199314,2022-09-15 07:17:02.431877
119546,59984109,59984265,59984607,489834.338880,489835.612787,489838.405582,4.066702,1.273907,2.792795,0.245899,0.313253,14.753970,2022-09-15 07:17:05.313193,2022-09-15 07:17:06.561764


# View quality of detections

In [12]:
%matplotlib widget

ext_plots = [
            ECG_Detections(ecg_stream, r_peaks),
            Resp_Detections(co2_stream, resp_cycles),
            ]

w = get_viewer(reader, stream_names=['ECG_II'], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# View Spreading Depolarizations + Heart Rate Features + Respi Rate features

In [13]:
%matplotlib widget

rate_bins_resp = np.arange(5, 30, 0.5) # cpm resolution
step_bins_ecg = 1 # bpm resolution

window_size_mins = 3 # minutes

ext_plots = [
            Spreading_depol_bipol2(reader.streams['EEG'], detections = events),
            Spectrogram_eeg(reader.streams['EEG'], chan_name = 'ECoGA3-ECoGA2', wsize = 20, lf = 1, hf = 45),
            Respi_Rate(resp_cycles, resp_wsize_in_mins = window_size_mins, ratio_sat = 4, rate_bins_resp=rate_bins_resp),
            Heart_Rate(r_peaks, step_bins_ecg = step_bins_ecg, hrv_wsize_in_mins =window_size_mins, ratio_sat = 3),
            ]

w = get_viewer(reader, stream_names=['ECG_II'], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

In [16]:
# event_index = 10 # 2 SD + no variability
# event_index = 43 # 1 SD + variablity
event_index = 49 # NUP + tachycardia + loss of variability

win_jitter = np.timedelta64(20, 'm')
time_ev = np.datetime64(events.loc[event_index,'StartTimeGMT'])
new_time = time_ev - win_jitter

print(events.loc[event_index,'Name'])
print(time_ev)
print(events.loc[event_index,'Duration'])

w.time_slider.update_time(new_time=new_time)

/tmp/ipykernel_774228/447842486.py:6: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  time_ev = np.datetime64(events.loc[event_index,'StartTimeGMT'])


NUP
2022-09-14T16:46:41.056000
52187.3


In [26]:
events

,Name,Description,StartTime,Duration,Target,EventDisplay,Source,StartTimeGMT
0,SD,None,2022-09-09 23:56:32.490,3806.2,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-09 21:56:32.490000+00:00
1,2:mydriase dt,None,2022-09-10 06:00:56.000,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS,2022-09-10 04:00:56+00:00
2,1:propofol,None,2022-09-10 06:24:46.233,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS,2022-09-10 04:24:46.233000+00:00
3,TDM,None,2022-09-10 07:56:50.279,2040.6,"EEG,Trends","{'@color': '#000000', '#text': 'Shade'}",User,2022-09-10 05:56:50.279000+00:00
4,SD,None,2022-09-10 14:34:24.651,872,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 12:34:24.651000+00:00
5,SD,None,2022-09-10 16:48:16.812,571.7,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 14:48:16.812000+00:00
6,SD,None,2022-09-10 18:50:23.445,582.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 16:50:23.445000+00:00
7,SD,None,2022-09-10 20:25:40.028,513.799,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 18:25:40.028000+00:00
8,SD,None,2022-09-10 21:24:27.666,658.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 19:24:27.666000+00:00
9,SD,None,2022-09-10 22:13:13.495,705.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User,2022-09-10 20:13:13.495000+00:00
